### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
movie_cast_schema = StructType(fields=[
                    StructField("movieId", IntegerType(), True),
                    StructField("personId", IntegerType(), True),
                    StructField("characterName", StringType(), True),
                    StructField("genderId", IntegerType(), True),
                    StructField("castOrder", IntegerType(), True)])

In [0]:
movie_cast_df = spark.read.json(f"{bronze_folder_path}/{v_file_date}/movie_cast.json", schema=movie_cast_schema, multiLine=True)
display(movie_cast_df)

### Paso 2 - Renombrar las columns y añadir nuevas columnas

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

movie_cast_final_df = movie_cast_df\
    .withColumnsRenamed({"movieId":"movie_id","personId":"person_id","characterName":"character_name","genderId":"gender_id","castOrder":"cast_order"})\
    .withColumns({"ingestion_time":current_timestamp(),"enviroment":lit(v_environment), "file_date":lit(v_file_date)})

In [0]:
display(movie_cast_final_df)

### Paso 3- Eliminar las columnas no deseadas

In [0]:
movie_cast_final_df = movie_cast_final_df.drop("gender_id", "cast_order")

In [0]:
#overwrite_partition(movie_cast_final_df, "movie_silver", "movie_cast")

In [0]:
merge_condicion = "tgt.movie_id = src.movie_id AND tgt.person_id = src.person_id AND tgt.file_date = src.file_date"
merge_delta_lake(movie_cast_final_df, "movie_silver", "movie_cast", silver_folder_path, merge_condicion, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.movie_cast
GROUP BY file_date;

In [0]:
%sql
SELECT * FROM movie_silver.movie_cast